<a href="https://colab.research.google.com/github/maxy808/CLIP_video/blob/frame-selection-xiaoyang/CLIP_video_frame6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone -b frame-selection-xiaoyang https://github.com/maxy808/CLIP_video.git
%cd CLIP_video

Cloning into 'CLIP_video'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 80 (delta 32), reused 38 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (80/80), 1.37 MiB | 3.40 MiB/s, done.
Resolving deltas: 100% (32/32), done.
/content/CLIP_video


In [2]:
# From CLIP
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install opencv-python boto3 requests pandas

/bin/bash: line 1: conda: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 116.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 8.4 MB/s eta 0:00:00


In [3]:
# Connet to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# # Comment if the file exits
# !wget -O /content/drive/MyDrive/Data/msrvtt_data.zip https://github.com/ArrowLuo/CLIP4Clip/releases/download/v0.0/msrvtt_data.zip
# !unzip /content/drive/MyDrive/Data/msrvtt_data.zip -d /content/drive/MyDrive/Data/

In [4]:
DATA_PATH = "/content/drive/MyDrive/Data/msrvtt_data"  # Directory of MSRVTT data
VIDEO_PATH = "/content/drive/MyDrive/Data/video"    # Directory of MSRVTT raw video

In [5]:
import pandas as pd

# Load training dataset
train_path = f"{DATA_PATH}/MSRVTT_train.9k.csv"
df = pd.read_csv(train_path)

print("Size: ", len(df))
print("Column names", df.columns.tolist())
display(df.head())

# Choose a subset
def sample_subset(df, frac=0.01, random_state=42):
    return df.sample(frac=frac, random_state=random_state).reset_index(drop=True)

# Set fraction to 0.01
subset_df = sample_subset(df, frac=0.01)
print("Size of subset：", len(subset_df))
display(subset_df.head())


Size:  9000
Column names ['video_id']


,video_id
0,video0
1,video1
2,video2
3,video3
4,video4


Size of subset： 90


,video_id
0,video8405
1,video1162
2,video582
3,video4081
4,video9139


In [6]:
# Save the subset
subset_path = f"{DATA_PATH}/MSRVTT_train.subset.csv"
subset_df.to_csv(subset_path, index=False)

print("Subset CSV saved to:", subset_path)

Subset CSV saved to: /content/drive/MyDrive/Data/msrvtt_data/MSRVTT_train.subset.csv


In [7]:
import torch
print("Current device:", torch.cuda.get_device_name() if torch.cuda.is_available() else "CPU only")

Current device: Tesla T4


In [8]:
# Download CLIP (ViT-B/32) weight
!wget -P ./modules https://openaipublic.azureedge.net/clip/models/cc12fdd5df8b6c2c9118e754d5f1ec50b6706b3cd1a1e0dabfdfd5fcd8c38e38/ViT-B-32.pt

--2025-04-19 03:30:58--  https://openaipublic.azureedge.net/clip/models/cc12fdd5df8b6c2c9118e754d5f1ec50b6706b3cd1a1e0dabfdfd5fcd8c38e38/ViT-B-32.pt
Resolving openaipublic.azureedge.net (openaipublic.azureedge.net)... 13.107.246.70, 2620:1ec:bdf::70
Connecting to openaipublic.azureedge.net (openaipublic.azureedge.net)|13.107.246.70|:443... connected.
HTTP request sent, awaiting response... 404 The specified blob does not exist.
2025-04-19 03:30:58 ERROR 404: The specified blob does not exist..



In [9]:
torch.cuda.empty_cache()

In [10]:
# Updata changes to files. Comment if the files are not changed
!cd CLIP_video && git pull

/bin/bash: line 1: cd: CLIP_video: No such file or directory


In [11]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

!python /content/CLIP_video/main_task_retrieval.py \
--do_train \
--num_thread_reader=0 \
--epochs=5 \
--batch_size=32 \
--train_csv {DATA_PATH}/MSRVTT_train.subset.csv \
--val_csv {DATA_PATH}/MSRVTT_JSFUSION_test.csv \
--data_path {DATA_PATH}/MSRVTT_data.json \
--features_path {VIDEO_PATH} \
--output_dir /content/CLIP_video/ckpts/ckpt_msrvtt_retrieval_looseType \
--lr 1e-4 \
--max_words 32 \
--max_frames 8 \
--batch_size_val 8 \
--datatype msrvtt \
--expand_msrvtt_sentences \
--feature_framerate 1 \
--coef_lr 1e-3 \
--freeze_layer_num 0 \
--slice_framepos 6 \
--loose_type \
--linear_patch 2d \
--sim_header meanP \
--pretrained_clip_name ViT-B/32 \
--fp16 --fp16_opt_level O1


04/19/2025 03:32:21 - INFO -   Effective parameters:
04/19/2025 03:32:21 - INFO -     <<< batch_size: 32
04/19/2025 03:32:21 - INFO -     <<< batch_size_val: 8
04/19/2025 03:32:21 - INFO -     <<< cache_dir: 
04/19/2025 03:32:21 - INFO -     <<< coef_lr: 0.001
04/19/2025 03:32:21 - INFO -     <<< cross_model: cross-base
04/19/2025 03:32:21 - INFO -     <<< cross_num_hidden_layers: 4
04/19/2025 03:32:21 - INFO -     <<< data_path: /content/drive/MyDrive/Data/msrvtt_data/MSRVTT_data.json
04/19/2025 03:32:21 - INFO -     <<< datatype: msrvtt
04/19/2025 03:32:21 - INFO -     <<< do_eval: False
04/19/2025 03:32:21 - INFO -     <<< do_lower_case: False
04/19/2025 03:32:21 - INFO -     <<< do_pretrain: False
04/19/2025 03:32:21 - INFO -     <<< do_train: True
04/19/2025 03:32:21 - INFO -     <<< epochs: 5
04/19/2025 03:32:21 - INFO -     <<< eval_frame_order: 0
04/19/2025 03:32:21 - INFO -     <<< expand_msrvtt_sentences: True
04/19/2025 03:32:21 - INFO -     <<< feature_framerate: 1
04/19/20